<a href="https://colab.research.google.com/github/BFvandy/Nissan-Text-Mining-POC-with-Transformer/blob/main/Reddit_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
cur_path = "/content/drive/My Drive/Nissan Internship/Reddit Comments"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/Nissan Internship/Reddit Comments


# Load Transformer

In [4]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.1 MB/s 
     |████████████████████████████████| 101 kB 15.5 MB/s 
     |████████████████████████████████| 596 kB 71.0 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

In [6]:
# example:
sequence_to_classify = "I purchased a new Civic in 2006, I knew the Honda quality ."
candidate_labels = ['I love Honda', 'I hate Honda', 'What about Honda']
classifier(sequence_to_classify, candidate_labels)

{'labels': ['I love Honda', 'What about Honda', 'I hate Honda'],
 'scores': [0.5322352051734924, 0.46602582931518555, 0.0017389195272698998],
 'sequence': 'I purchased a new Civic in 2006, I knew the Honda quality .'}

# Adjust Dataframe Structure

In [7]:
comments = pd.read_csv('reddit_all_preprocessed.csv')
reddit_cmt_fun = comments.copy()
reddit_cmt_pers = comments.copy() 

In [8]:
fun_labels = ['Dependable', 'Lasts long', 'Value for money', 'Quality fit and finish', 'Attractive styling'
                                    , 'Safe', 'Retains resale value', 'Driver comfort', 'Fun to drive', 
                                    'Advanced features', 'Responsive handling', 'Prestigious', 'Dealerships', 
        'Fuel efficient', 'Quick acceleration', 'Environmentally friendly', 'Affordable']
pers_labels = ['Trusted', 'Leader', 'Responsible', 'Confident', 'Innovative', 'Exciting', 'Practical', 'Adventurous', 
               'Passionate', 'Distinctive', 'Youthful', 'Aggressive']

fun_labels_lower = [i.lower() for i in fun_labels]
pers_labels_lower= [i.lower() for i in pers_labels]

In [9]:
# data structure
reddit_cmt_fun[fun_labels_lower] = pd.NaT
reddit_cmt_fun.head(5)

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable
0,Nissan,Rogue,Reddit,this is incredible timing for me as my cvt jus...,1.659024e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Nissan,Rogue,Reddit,op do you own it out right? i'd sell it or tra...,1.659036e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Nissan,Rogue,Reddit,i have 2019 sport transmission failed at 42k m...,1.659033e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Nissan,Rogue,Reddit,not yet. but i bought an extended warranty any...,1.659030e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Nissan,Rogue,Reddit,my 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [10]:
reddit_cmt_pers[pers_labels_lower] = pd.NaT
reddit_cmt_pers.head(5)

,Make,Model,Source,Text,timestamp,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Rogue,Reddit,this is incredible timing for me as my cvt jus...,1.659024e+09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Nissan,Rogue,Reddit,op do you own it out right? i'd sell it or tra...,1.659036e+09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Nissan,Rogue,Reddit,i have 2019 sport transmission failed at 42k m...,1.659033e+09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Nissan,Rogue,Reddit,not yet. but i bought an extended warranty any...,1.659030e+09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Nissan,Rogue,Reddit,my 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


# Apply

In [11]:
import time

start = time.time()

for i in range(len(reddit_cmt_fun)):
  try:
    res = classifier(reddit_cmt_fun.at[i, 'Text'].lower(), fun_labels_lower, multi_label=True)
    reddit_cmt_fun.at[i, fun_labels_lower] = dict(zip(res['labels'], res['scores']))
  except:
    pass

end = time.time()
print((end - start)/60)

0.09354063669840494


In [19]:
reddit_cmt_fun.head(10)

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable
0,Nissan,Rogue,Reddit,this is incredible timing for me as my cvt jus...,1.659024e+09,0.001459,0.046492,0.836753,0.018161,0.331459,...,0.09187,0.311208,0.166514,0.938747,0.756797,0.023775,0.004544,0.638233,0.009812,0.219503
1,Nissan,Rogue,Reddit,op do you own it out right? i'd sell it or tra...,1.659036e+09,0.002094,0.790318,0.645161,0.034802,0.333479,...,0.526348,0.051011,0.409513,0.578997,0.178425,0.126561,0.063034,0.309499,0.189067,0.684214
2,Nissan,Rogue,Reddit,i have 2019 sport transmission failed at 42k m...,1.659033e+09,0.086808,0.022779,0.107647,0.034809,0.630758,...,0.62545,0.147929,0.689673,0.159978,0.119343,0.544357,0.17782,0.540098,0.144392,0.497203
3,Nissan,Rogue,Reddit,not yet. but i bought an extended warranty any...,1.659030e+09,0.314663,0.22102,0.830511,0.113424,0.466922,...,0.639512,0.30101,0.253473,0.357972,0.529661,0.21291,0.105282,0.138708,0.135744,0.712273
4,Nissan,Rogue,Reddit,my 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,0.00241,0.015241,0.232259,0.000322,0.038496,...,0.487458,0.010045,0.013138,0.014472,0.24327,0.007173,0.00132,0.080628,0.01717,0.248629
5,Nissan,Rogue,Reddit,"2015 altima . 80k, cvt still strong.",1.659045e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,Nissan,Rogue,Reddit,"yep, too bad i'm past the mileage already",1.659056e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,Nissan,Rogue,Reddit,cvt needs to go extinct,1.659062e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,Nissan,Rogue,Reddit,"i should've god 1500 warranty for ""100k miles""...",1.659056e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
9,Nissan,Rogue,Reddit,i have a 2003 xterra with a lil less than 150k...,1.659062e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
